In [1]:
from solve_LMI import solve_LMI
from solve_LP import solve_LP
import numpy as np
from scipy.linalg import toeplitz
from circulant_embedding import faster_A
from scipy.sparse import diags
from SCGAL import run_solver
import time
import json

In [2]:
n = 1
max_runs = 100

starting_d = 1
ending_d = 5
ds = list(range(starting_d, ending_d+1))
circ_emb_obj = np.zeros((max_runs, len(ds)))
circ_emb_time = np.zeros((max_runs, len(ds)))
LMI_obj = np.zeros((max_runs, len(ds)))
LMI_time = np.zeros((max_runs, len(ds)))
SCGAL_obj = np.zeros((max_runs, len(ds)))
SCGAL_time = np.zeros((max_runs, len(ds)))
cs = np.random.normal(size=(100, 4*ds[-1], len(ds)))

# d = ds[0]
# c = cs[:4*d, 0]

In [3]:
# Circulant embedding
for i in range(len(ds)):
    d = ds[i]

    A = faster_A(n, d)
    A_0 = np.ones(np.shape(A)[0])

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LP, optimal_value_LP, optimal_vars_LP = solve_LP(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        circ_emb_obj[num_iterations, i] = optimal_value_LP
        circ_emb_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)
        
        # print("LP Status:", status_LP)
        # print("LP Optimal Value:", optimal_value_LP)
        # print("LP Optimal Variables:", [var.value for var in optimal_vars_LP])

1 0.009186979965306818
1 0.012464607949368656
1 0.015194197942037135
1 0.01757413090672344
1 0.020341817929875106
1 0.023234390886500478
1 0.0257045128964819
1 0.028009198897052556
1 0.0305780359194614
1 0.032855559955351055
1 0.035011598956771195
1 0.03713784791762009
1 0.03920220991130918
1 0.041342203912790865
1 0.04350122791947797
1 0.045637737901415676
1 0.04784335888689384
1 0.04994562192587182
1 0.0521251538884826
1 0.05418585689039901
1 0.05621795490151271
1 0.05846979294437915
1 0.06055864994414151
1 0.06263262598076835
1 0.06472608697367832
1 0.0667925049783662
1 0.06889778899494559
1 0.07103900803485885
1 0.0730660289991647
1 0.07510951603762805
1 0.07712853007251397
1 0.07919419009704143
1 0.08135211310582235
1 0.08338799013290554
1 0.08541002217680216
1 0.0874394581769593
1 0.08948250015964732
1 0.09155485313385725
1 0.09353333013132215
1 0.09558186016511172
1 0.09758500719908625
1 0.09961637516971678
1 0.10182112915208563
1 0.10383037716383114
1 0.10587358614429832
1 0.10

In [4]:
# LMI formulation with toeplitz constraints
for i in range(len(ds)):
    d = ds[i]

    col = np.zeros(2*d + 1, dtype=complex)
    col[0] = 1
    A_0 = toeplitz(col)

    A = []

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = 1
        mx = toeplitz(col)
        A.append(mx)

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = -1j
        mx = toeplitz(col)
        A.append(mx)

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LMI, optimal_value_LMI, optimal_vars_LMI = solve_LMI(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        LMI_obj[num_iterations, i] = optimal_value_LMI
        LMI_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Toeplitz LMI Status:", status_LMI)
    # print("Toeplitz LMI Optimal Value:", optimal_value_LMI)
    # print("Toeplitz LMI Optimal Variables:", [var.value for var in optimal_vars_LMI])

1 0.012953724013641477
1 0.01983157201902941
1 0.025598396023269743
1 0.03086238499963656
1 0.03588706499431282
1 0.04080718499608338
1 0.0456105989869684
1 0.050562524993438274
1 0.055376884003635496
1 0.060304329032078385
1 0.06534189102239907
1 0.07005208102054894
1 0.07490928401239216
1 0.07982664904557168
1 0.08494411804713309
1 0.0897724560345523
1 0.09455420798622072
1 0.09935486497124657
1 0.10420087992679328
1 0.10889858490554616
1 0.11368986091110855
1 0.11856763588730246
1 0.1236151588964276
1 0.128500469843857
1 0.13323356781620532
1 0.13813712686533108
1 0.14360325987217948
1 0.14875540288630873
1 0.15406067291041836
1 0.1599844919401221
1 0.1650930799660273
1 0.1701671159826219
1 0.175177232013084
1 0.17994198302039877
1 0.1846205519977957
1 0.18961795000359416
1 0.1944487449945882
1 0.19922589999623597
1 0.20409366901731119
1 0.20897094602696598
1 0.21365445107221603
1 0.21829486708156765
1 0.22436457307776436
1 0.22978469007648528
1 0.23480171407572925
1 0.2408118470921

In [5]:
# SPARSE LMI formulation for Sketchy CGAL
for i in range(len(ds)):
    d = ds[i]

    diag = np.ones(2*d + 1, dtype=complex)
    A_0 = diags(diag)

    A = []

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([diag, diag],offsets=[-m,m])
        A.append(mx)

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([-1j*diag, 1j*diag],offsets=[-m,m])
        A.append(mx)

    A_norm = 2*d

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        U, Lambda, objective, Omega, z, y, S = run_solver(A_0, A, c, 2*d + 1, 4*d,
                                                        alpha=2*np.linalg.norm(c,ord=2), A_norm=A_norm,
                                                        R=1, T=5000,
                                                        trace_mode='bound')
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        SCGAL_obj[num_iterations, i] = -objective
        SCGAL_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Calculated Objective:", -objective)

1 0.1973280850215815
1 0.38479778601322323
1 0.652602243993897
1 0.9828899699496105
1 1.1732377429725602
1 1.3986779029364698
1 1.5897671949351206
1 1.9693545369664207
1 2.2738705209922045
1 2.5233373179798946
1 2.721079701965209
1 2.8939949249615893
1 3.1138304349733517
1 3.3435096569592133
1 3.603278923954349
1 3.790248362987768
1 4.024414280953351
1 4.220406901964452
1 4.482702212990262
1 4.734445516951382
1 4.879525088996161
1 5.160663582966663
1 5.413250388926826
1 5.627584106929135
1 5.837937256903388
1 6.062056328926701
1 6.358500650909264
1 6.679686680901796
1 6.948524845938664
1 7.185872994945385
1 7.504247622913681
1 7.710445152944885
1 8.223610402958002
1 8.441273413947783
1 8.592978638946079
1 8.853447759989649
1 9.190818579983898
1 9.380711310019251
1 9.533650282013696
1 9.792065960005857
1 9.99742095102556
1 10.18347651703516
1 10.466348136018496
1 10.870728262001649
1 11.07003005797742
1 11.525533499021549
1 11.669990157999564
1 11.90552403399488
1 12.161727697995957
1 1

In [6]:
my_dict = {'ds': list(ds), 'circ_emb_obj': circ_emb_obj.tolist(), 'circ_emb_time': circ_emb_time.tolist(),
           'LMI_obj': LMI_obj.tolist(), 'LMI_time': LMI_time.tolist(),
           'SCGAL_obj': SCGAL_obj.tolist(), 'SCGAL_time': SCGAL_time.tolist()}
# Save the results to a JSON file
file_name = 'ds' + str(ds[0]) + 'to' + str(ds[-1]) + '.json'
with open(file_name, 'w') as f:
    json.dump(my_dict, f)

In [7]:
with open(file_name, 'r') as f:
    loaded_dict = json.load(f)

In [8]:
ds_loaded = np.array(loaded_dict['ds'])
circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
LMI_time_loaded = np.array(loaded_dict['LMI_time'])
SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])

In [9]:
assert np.allclose(ds, ds_loaded)
assert np.allclose(circ_emb_obj, circ_emb_obj_loaded)
assert np.allclose(circ_emb_time, circ_emb_time_loaded)
assert np.allclose(LMI_obj, LMI_obj_loaded)
assert np.allclose(LMI_time, LMI_time_loaded)
assert np.allclose(SCGAL_obj, SCGAL_obj_loaded)
assert np.allclose(SCGAL_time, SCGAL_time_loaded)

In [15]:
print(circ_emb_obj_loaded[3, 0])
print(LMI_obj_loaded[3, 0])
print(SCGAL_obj_loaded[3, 0])

-2.7627660451053573
-2.862591624147788
-2.961063044414708


In [10]:
# start_idx = [1, 6, 11, 16]
# end_idx = [5, 10, 15, 20]

# for i in range(len(start_idx)):
#     file_name = 'ds' + str(start_idx[i]) + 'to' + str(end_idx[i]) + '.json'

#     with open(file_name, 'r') as f:
#         loaded_dict = json.load(f)

#     if i == 0:
#         ds_loaded = np.array(loaded_dict['ds'])
#         circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
#         circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
#         LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
#         LMI_time_loaded = np.array(loaded_dict['LMI_time'])
#         SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
#         SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])
#     else:
#         ds_loaded = np.concatenate((ds_loaded, np.array(loaded_dict['ds'])), axis=0)
#         circ_emb_obj_loaded = np.concatenate((circ_emb_obj_loaded, np.array(loaded_dict['circ_emb_obj'])), axis=1)
#         circ_emb_time_loaded = np.concatenate((circ_emb_time_loaded, np.array(loaded_dict['circ_emb_time'])), axis=1)
#         LMI_obj_loaded = np.concatenate((LMI_obj_loaded, np.array(loaded_dict['LMI_obj'])), axis=1)
#         LMI_time_loaded = np.concatenate((LMI_time_loaded, np.array(loaded_dict['LMI_time'])), axis=1)
#         SCGAL_obj_loaded = np.concatenate((SCGAL_obj_loaded, np.array(loaded_dict['SCGAL_obj'])), axis=1)
#         SCGAL_time_loaded = np.concatenate((SCGAL_time_loaded, np.array(loaded_dict['SCGAL_time'])), axis=1)


# ds_loaded = ds_loaded[:17]
# circ_emb_obj_loaded = circ_emb_obj_loaded[:, :17]
# circ_emb_time_loaded = circ_emb_time_loaded[:, :17]
# LMI_obj_loaded = LMI_obj_loaded[:, :17]
# LMI_time_loaded = LMI_time_loaded[:, :17]
# SCGAL_obj_loaded = SCGAL_obj_loaded[:, :17]
# SCGAL_time_loaded = SCGAL_time_loaded[:, :17]
    